# Stage 2: Data Annotation

In this notebook, I'll leverage a Large Language Model (LLM) to perform sentiment annotation on the ESG document dataset, assigning scores of 0 for negative, 0.5 for neutral, and 1 for positive sentiment.  
The workflow involves manually creating a "gold standard" by annotating ~500 sentences, afterward setting up 2-3 LLMs for trial annotations, and experimenting with prompting strategies (zero-shot/few-shot) that we'll evaluate against the "gold standard".

## Setup & Data Loading

In [3]:
# Imports
import os
import ast
import pandas as pd
import numpy as np

In [4]:
# Load the preprocessed data
cleaned_data = pd.read_csv('../data/checkpoints/enriched_cleaned_data.csv', delimiter = '|')

In [5]:
# Define a function to convert a string representation of a list to a list datatype
def string_to_list(string):
    try:
        return ast.literal_eval(string)
    except (ValueError, SyntaxError):
        print('List conversion failed')
        return []

# Convert the string representations of the lists to the correct 'list' datatype
cleaned_data['word_tokens'] = cleaned_data['word_tokens'].apply(string_to_list)
cleaned_data['sentence_tokens'] = cleaned_data['sentence_tokens'].apply(string_to_list)
cleaned_data['pos_tagged_word_tokens'] = cleaned_data['pos_tagged_word_tokens'].apply(string_to_list)
cleaned_data['pos_tagged_sentence_tokens'] = cleaned_data['pos_tagged_sentence_tokens'].apply(string_to_list)
cleaned_data['esg_topics'] = cleaned_data['esg_topics'].apply(string_to_list)

In [6]:
# Add some count features for the analysis
cleaned_data['cnt_word'] = cleaned_data['word_tokens'].apply(len)
cleaned_data['cnt_sentence'] = cleaned_data['sentence_tokens'].apply(len)
cleaned_data['cnt_esg'] = cleaned_data['esg_topics'].apply(len)

# Calculate ratio between words/sentences
cleaned_data['ratio_word_sentence'] = cleaned_data['cnt_word'] / cleaned_data['cnt_sentence']

# Convert date to correct datatype
cleaned_data['date'] = pd.to_datetime(cleaned_data['date'])

# Derive year and month to aggregate
cleaned_data['year_month'] = cleaned_data['date'].apply(lambda x: x.strftime('%Y-%m'))
cleaned_data['year'] = cleaned_data['date'].apply(lambda x: x.strftime('%Y'))
cleaned_data['month'] = cleaned_data['date'].apply(lambda x: x.strftime('%m'))

In [7]:
# Define function to save intermediary steps in a file
def csv_checkpoint(df, filename='checkpoint'):
    """
    Saves a DataFrame to a CSV file and loads it back into a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame to save and load.
        filename (str): The name of the CSV file to save the DataFrame to (default: 'checkpoint').

    Returns:
        pandas.DataFrame: The loaded DataFrame.
    """
    if not os.path.exists('../data/checkpoints/'):  # Check if the directory exists and create it if it doesn't
        os.makedirs('../data/checkpoints/')

    # Save DataFrame to CSV
    df.to_csv(f'../data/checkpoints/{filename}.csv', index=False, sep='|')  # Save DataFrame to CSV with specified filename
    print(f'Saved DataFrame to {filename}.csv')

    # Load CSV back into DataFrame
    df = pd.read_csv(f'../data/checkpoints/{filename}.csv', delimiter='|')  # Load CSV back into DataFrame
    print(f'Loaded DataFrame from {filename}.csv')

    return df

## Manual sentence sentiment annotation

To define a "gold standard" for the sentiment, 500 randomly sampled sentences are manually annotate with:  
**0 = negative, 0.5 = neutral, 1 = positive**

In [117]:
# Crate a deep copy so no reload from CSV files is necessary
documents = cleaned_data.copy(deep=True)

In [118]:
# Craete new column to store the sentence sentiment
documents['sentence_sentiment_value_llm'] = np.nan

In [119]:
# Explode the dataset based on the sentence tokens, so each row contains one sentence
documents = documents.explode('sentence_tokens')

# Preserve original index, so a later aggregation is possible
documents['original_index'] = documents.index

# Reset the index
documents = documents.reset_index(drop=True)

In [120]:
# Separate the DataFrame into internal/external sentences with a defined ratio
internal = documents[documents['internal'] == 1]
external = documents[documents['internal'] == 0]

# Determine the number of samples from each group, 1000 sentences in total
n_internal = int(0.2 * 1000)  # 20% of samples
n_external = 1000 - n_internal  # Remaining samples

# Sample 1000 random sentences with a seed, so a re-run samples the same sentences
sampled_internal = internal.sample(n=n_internal, random_state=42)
sampled_external = external.sample(n=n_external, random_state=42)

# Concatenate and shuffle the samples the DataFrames
sampled_documents = pd.concat([sampled_internal, sampled_external])
sampled_documents = sampled_documents.sample(frac=1, random_state=42)

# Drop the sampled rows from the original DataFrame
documents = documents.drop(sampled_documents.index)

In [134]:
# Check the sampled data
sampled_documents[['title','sentence_tokens','internal','sentence_sentiment_value_llm']].head(10)

,title,sentence_tokens,internal,sentence_sentiment_value_llm
529113,Transcript levels in plasma contribute substan...,therefore adjust differences sample quality in...,0,0.5
339673,Absolutely everything you need to go bikepacki...,way little quicker easier make coffee porridge...,0,0.5
390711,STARTUP STAGE: Tripshifu connects experienced ...,started career multinational tata steel joinin...,0,0.5
354554,Automotive Aftermarket Market by Global Busine...,notable trend currently influencing dynamics a...,0,0.5
420428,Smashing Podcast Episode 50 With Marko Dugonji...,know never used tables layout,0,1.0
457123,TSMC Considers EMEA and APAC Supply Chain Expa...,learned anything first half pandemic globalisa...,0,1.0
669146,Ukelele revival creates the Magic Fluke,considered cool instrument among younger people,0,1.0
452818,Financial Inclusion Is Nothing Without Securit...,prepaid cards convenience means money already ...,0,1.0
265897,Tony Fernandes Steps Down As AirAsia X Group CEO,keen amateur photographer also recently reache...,0,1.0
236198,BeiersdorfAG Sustainability Report 2020,therefore constantly optimizing disposal chann...,1,1.0


In [ ]:
# Loop the samples to annotate them
for idx, row in sampled_documents.iterrows():
    # Loop until valid input is received
    while True:
        # Print the title of the document and the sentence
        print(f"Title: {row['title']}\nSentence: {row['sentence_tokens']}\n")

        # Wait for user input
        sentiment = input("Enter sentiment value (+ for 1.0, - for 0.0, Enter for 0.5): ")

        # Check if the input is valid
        if sentiment == '+':
            sampled_documents.at[idx, 'sentence_sentiment_value_llm'] = 1.0
            break
        elif sentiment == '-':
            sampled_documents.at[idx, 'sentence_sentiment_value_llm'] = 0.0
            break
        elif sentiment == '':
            sampled_documents.at[idx, 'sentence_sentiment_value_llm'] = 0.5
            break
        else:
            print("Invalid input. Please try again.")

In [138]:
# Check the manual annotations
sampled_documents[sampled_documents['sentence_sentiment_value_llm'].notnull()].head(10)

,company,datatype,date,domain,esg_topics,internal,symbol,title,cleaned_content,word_tokens,...,sentiment_value,cnt_word,cnt_sentence,cnt_esg,ratio_word_sentence,year_month,year,month,sentence_sentiment_value_llm,original_index
529113,Qiagen,thinktank,2022-03-17,thelancet,"[GenderDiversity, Privacy]",0,QIA,Transcript levels in plasma contribute substan...,aa remain underrepresented alzheimers disease ...,"[aa, remain, underrepresented, alzheimers, dis...",...,0.100772,5054,280,2,18.050000,2022-03,2022,03,0.5,7601
339673,Beiersdorf,general,2021-04-27,cyclingweekly,"[Compliance, Recycling, CustomerService, Gende...",0,BEI,Absolutely everything you need to go bikepacki...,get know area far intimately staying accommoda...,"[get, know, area, far, intimately, staying, ac...",...,0.314033,5031,431,4,11.672854,2021-04,2021,04,0.5,2774
390711,Deutsche Bank,general,2022-09-26,phocuswire,"[HumanCapital, Social, Recruiting, Misinformat...",0,DBK,STARTUP STAGE: Tripshifu connects experienced ...,founded february currently five employee idea ...,"[founded, february, currently, five, employee,...",...,0.249278,520,50,4,10.400000,2022-09,2022,09,0.5,4228
354554,Continental,general,2021-10-05,ecochunk,[RussianFederation],0,CON,Automotive Aftermarket Market by Global Busine...,recording estimating analysing market data rep...,"[recording, estimating, analysing, market, dat...",...,0.280167,676,36,1,18.777778,2021-10,2021,10,0.5,3046
420428,Deutsche Telekom,business,2022-08-09,smashingmagazine,"[CorporateCulture, HumanCapital, Environment, ...",0,DTE,Smashing Podcast Episode 50 With Marko Dugonji...,ask affect change ux design large organization...,"[ask, affect, change, ux, design, large, organ...",...,0.203908,2392,320,4,7.475000,2022-08,2022,08,1.0,4946
457123,Infineon Technologies,tech,2021-07-26,supplychaindigital,[Cybersecurity],0,IFX,TSMC Considers EMEA and APAC Supply Chain Expa...,tsmc industry leader chipmaking renowned globa...,"[tsmc, industry, leader, chipmaking, renowned,...",...,0.253957,453,23,1,19.695652,2021-07,2021,07,1.0,6085
669146,Volkswagen,general,2023-03-13,ctinsider,[Grantmaking],0,VOW3,Ukelele revival creates the Magic Fluke,use next previous button navigate show variety...,"[use, next, previous, button, navigate, show, ...",...,0.151402,874,102,1,8.568627,2023-03,2023,03,1.0,10626
452818,Infineon Technologies,business,2021-05-16,thefintechtimes,"[Fraud, DataSecurity, UnbankedPopulation, Incl...",0,IFX,Financial Inclusion Is Nothing Without Securit...,onus card issuer bank drive trajectory ceo ide...,"[onus, card, issuer, bank, drive, trajectory, ...",...,0.302295,523,41,5,12.756098,2021-05,2021,05,1.0,5937
265897,Airbus,tech,2022-11-01,simpleflying,[Corruption],0,AIR,Tony Fernandes Steps Down As AirAsia X Group CEO,first flight tomorrow mark since airasia first...,"[first, flight, tomorrow, mark, since, airasia...",...,0.200262,290,26,1,11.153846,2022-11,2022,11,1.0,942
236198,Beiersdorf,sustainability_report,2020-03-31,NaN,"[RoundtableOnSustainablePalmOil, CleanWater, D...",1,BEI,BeiersdorfAG Sustainability Report 2020,commitments08 overview consumer business segme...,"[commitments08, overview, consumer, business, ...",...,0.335896,10031,604,51,16.607616,2020-03,2020,03,1.0,89


In [139]:
documents[documents['sentence_sentiment_value_llm'].notnull()].head(10)

,title,company,datatype,date,domain,esg_topics,internal,symbol,cleaned_content,word_tokens,...,industry,sentiment_value,cnt_word,cnt_sentence,cnt_esg,ratio_word_sentence,year_month,year,month,sentence_sentiment_value_llm
0,BeiersdorfAG Sustainability Report 2021,Beiersdorf,sustainability_report,2021-03-31,None,"[CleanWater, GHGEmission, ProductLiability, Va...",1,BEI,brand strategy sustainability agenda care beyo...,"[brand, strategy, sustainability, agenda, care...",...,Household & Personal Products,0.398557,4877,309,36,15.783172,2021-03,2021,03,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,DeutscheTelekomAG Sustainability Report 2021,Deutsche Telekom,sustainability_report,2021-03-31,None,"[DataSecurity, Iso50001, GlobalWarming, Produc...",1,DTE,management fact deutsche telekoms cr report th...,"[management, fact, deutsche, telekoms, cr, rep...",...,Telecom Services,0.204224,53878,4379,102,12.303722,2021-03,2021,03,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,VonoviaSE Sustainability Report 2021,Vonovia,sustainability_report,2021-03-31,None,"[Whistleblowing, DataSecurity, Vaccine, GHGEmi...",1,VNA,sustainable future sustainability report dear ...,"[sustainable, future, sustainability, report, ...",...,Real Estate Services,0.241932,36507,2477,73,14.738393,2021-03,2021,03,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,MerckKGaA Sustainability Report 2021,Merck,sustainability_report,2021-03-31,None,"[DataSecurity, DataMisuse, DrugResistance, Iso...",1,MRK,management employee profile attractive employe...,"[management, employee, profile, attractive, em...",...,Drug Manufacturers—Specialty & Generic,0.235490,46497,3215,127,14.462519,2021-03,2021,03,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,MTUAeroEngines Sustainability Report 2020,MTU,sustainability_report,2020-03-31,None,"[WorkLifeBalance, Corruption, AirQuality, Data...",1,MTX,sustainability go far beyond climate action sa...,"[sustainability, go, far, beyond, climate, act...",...,Aerospace & Defense,0.241814,21570,1521,82,14.181460,2020-03,2020,03,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
5,E.ONSE Sustainability Report 2021,E ONSE,sustainability_report,2021-03-31,None,"[DataSecurity, Iso50001, GlobalWarming, Employ...",1,EOAN,standwithukraine sustainability report search ...,"[standwithukraine, sustainability, report, sea...",...,Utilities—Diversified,0.221860,37805,2731,96,13.842915,2021-03,2021,03,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
6,RWEAG Sustainability Report 2021,RWE,sustainability_report,2021-03-31,None,"[WorkLifeBalance, Corruption, Iso50001, GHGEmi...",1,RWE,sustainability report energy sustainable life ...,"[sustainability, report, energy, sustainable, ...",...,Utilities—Diversified,0.201995,32497,2248,84,14.455961,2021-03,2021,03,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
7,HeidelbergCementAG Annual Report 2021,Heidelberg Cement,annual_report,2021-03-31,None,"[WorkLifeBalance, Vaccine, DataSecurity, GHGEm...",1,HEI,tonne aggregate tonne readymixed concrete cubi...,"[tonne, aggregate, tonne, readymixed, concrete...",...,Building Materials,0.208050,69720,4161,74,16.755588,2021-03,2021,03,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
8,HeidelbergCementAG Sustainability Report 2020,Heidelberg Cement,sustainability_report,2020-03-31,None,"[CleanWater, Corruption, Whistleblowing, AntiC...",1,HEI,business product production employee society c...,"[business, product, production, employee, soci...",...,Building Materials,0.229841,21116,1251,79,16.879297,2020-03,2020,03,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
9,SiemensAG Sustainability Report 2020,Siemens,sustainability_report,2020-03-31,None,"[DataSecurity, Iso50001, EmployeeTurnover, Wat...",1,SIE,environment social annex glance sustainability...,"[environment, social, annex, glance, sustainab...",...,Specialty Industrial Machinery,0.268232,31996,2030,97,15.761576,2020-03,2020,03,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [140]:
documents = pd.concat([documents, sampled_documents])

In [130]:
# Group by the 'original_index' and aggregate the columns
documents = documents.groupby('original_index').agg({
    # Use 'first' function for all columns except for 'sentence_tokens' and 'sentence_sentiment_value_llm'
    'title': 'first',  
    'company': 'first',  
    'datatype': 'first',  
    'date': 'first',  
    'domain': 'first',  
    'esg_topics': 'first',  
    'internal': 'first',  
    'symbol': 'first',  
    'title': 'first',  
    'cleaned_content': 'first',  
    'word_tokens': 'first',  
    'sentence_tokens': list,  # Combine the 'sentence_tokens' into a list
    'sentence_tokens': 'first',  
    'pos_tagged_word_tokens': 'first',  
    'pos_tagged_sentence_tokens': 'first',  
    'market_cap_in_usd_b': 'first',  
    'sector': 'first',  
    'industry': 'first',  
    'sentiment_value': 'first',  
    'cnt_word': 'first',  
    'cnt_sentence': 'first',  
    'cnt_esg': 'first',  
    'ratio_word_sentence': 'first',  
    'year_month': 'first',  
    'year': 'first',  
    'month': 'first',  
    'sentence_sentiment_value_llm': lambda x: [i if pd.notnull(i) else np.nan for i in x]  # Combine the 'sentence_sentiment_value_llm' into a list, substituting NaN where no manual sentiment was added
})

# Reset the index
documents = documents.reset_index(drop=True)

In [109]:
# Use ChatGTP API to annoatete with zero-shot or few-show prompts:

,title,sentence_tokens
0,BeiersdorfAG Sustainability Report 2021,brands strategy sustainability agenda care bey...
